<a href="https://colab.research.google.com/github/Az-Ks/AirQo-Ugandan-Air-Quality-Forecast/blob/master/XGBOOST_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **PLEASE CHENGE YOUR RUNTIME TYPE TO GPU **

In [0]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import math
import gc

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
path =  '/content/drive/My Drive/'

In [0]:
train=pd.read_csv("/content/drive/My Drive/Train (14).csv")
test=pd.read_csv("/content/drive/My Drive/Test (8).csv")

### Features engineering part

In [0]:
# covert features  fron string to List of values 
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    train[feature]=train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    test[feature]=test[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])    

In [0]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    #x["var_"+col_name]=x[col_name].apply(np.var)
    x["sum_"+col_name]=x[col_name].apply(np.sum)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  
def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]

In [0]:
data=pd.concat([train,test],sort=False).reset_index(drop=True)
data.columns.tolist()

['ID',
 'location',
 'temp',
 'precip',
 'rel_humidity',
 'wind_dir',
 'wind_spd',
 'atmos_press',
 'target']

In [0]:
for x in range(121):
    data["newtemp"+ str(x)] = data.temp.str[x]
    data["newprecip"+ str(x)] = data.precip.str[x]
    data["newrel_humidity"+ str(x)] = data.rel_humidity.str[x]
    data["newwind_dir"+ str(x)] = data.wind_dir.str[x]
    data["windspeed"+ str(x)] = data.wind_spd.str[x]
    data["atmospherepressure"+ str(x)] = data.atmos_press.str[x]

In [0]:
data.shape

(20574, 735)

In [0]:
len(data.precip[1])

121

In [0]:
for col_name in tqdm(features):
    data[col_name]=data[col_name].apply(remove_nan_values)

100%|██████████| 6/6 [00:02<00:00,  2.45it/s]


In [0]:
for col_name in tqdm(features):
    data=aggregate_features(data,col_name)

100%|██████████| 6/6 [00:14<00:00,  2.45s/it]


In [0]:
data.drop(features,1,inplace=True)

In [0]:
hum_features =  list(data.filter(regex='rel_humidity.*').columns)
temp_features = list( data.filter(regex='newtemp.*').columns)  
precip_features =  list(data.filter(regex='newprecip*').columns)
winddir_features = list( data.filter(regex='newwind_dir.*').columns)
windspead_features = list( data.filter(regex='windspeed.*').columns)
atm_features =  list(data.filter(regex='atmos.*').columns)




hum_features= hum_features[36:]
temp_features=temp_features[36:] 
precip_features=precip_features[31:]
winddir_features=winddir_features[36:]
windspead_features=windspead_features[36:]
atm_features=atm_features[36:]





data[hum_features]= data[hum_features].apply(lambda x: x.fillna(x.mean()),axis=1)



data[temp_features]= data[temp_features].apply(lambda x: x.fillna(x.mean()),axis=1)


data[precip_features]= data[precip_features].apply(lambda x: x.fillna(float(0.0)),axis=1)



data[winddir_features]= data[winddir_features].apply(lambda x: x.fillna(x.mean()),axis=1)



data[windspead_features]= data[windspead_features].apply(lambda x: x.fillna(x.mean()),axis=1)




data[atm_features]= data[atm_features].apply(lambda x: x.fillna(x.mean()),axis=1)


In [0]:
train=data[data.target.notnull()].reset_index(drop=True)
test=data[data.target.isna()].reset_index(drop=True)

In [0]:
train.shape,test.shape

((15539, 765), (5035, 765))

###end of la3b

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split
import requests
from io import StringIO 
%matplotlib inline
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split
import requests
from io import StringIO 
%matplotlib inline
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

In [0]:
test.drop("target",axis  = 1,inplace  = True)
train.shape,test.shape

((15539, 765), (5035, 764))

In [0]:
from sklearn.preprocessing import LabelEncoder
lab  = LabelEncoder()
lab.fit(train["location"])
train.location  = lab.transform(train.location)
test.location  = lab.transform(test.location)
train.target.min(),train.target.max()

(1.4526190476190477, 475.82)

In [0]:
X=train.drop(["ID",'target'],axis =1)
y = train.target
test_id  = test['ID']

In [0]:
from sklearn.cluster import KMeans
X=X.fillna(-9999)

kmeans = KMeans(n_clusters=6).fit(X)
X['cluster']=kmeans.predict(X)
test=test.fillna(-9999)
test['cluster']=kmeans.predict(test.drop(["ID"],axis = 1))




In [0]:
air_temperature_filler = pd.DataFrame(X.groupby(['location','mean_wind_dir'])['mean_temp'].mean())
X['mean temp per month']=0
for i in range (len(X)):
  
  X['mean temp per month'][i]=air_temperature_filler.loc[(X['location'][i],X['mean_wind_dir'][i]), :]

air_temperature_filler = pd.DataFrame(test.groupby(['location','mean_wind_dir'])['mean_temp'].mean())

test['mean temp per month']=0
for i in range (len(test)):
  
  test['mean temp per month'][i]=air_temperature_filler.loc[(test['location'][i],test['mean_wind_dir'][i]), :]

In [0]:
p0=['newtemp0', 'newprecip0', 'newrel_humidity0', 'newwind_dir0',
       'windspeed0', 'atmospherepressure0']
p4=['newtemp30', 'newprecip30', 'newrel_humidity30', 'newwind_dir30',
       'windspeed30', 'atmospherepressure30']         
p1=['newtemp60', 'newprecip60', 'newrel_humidity60', 'newwind_dir60',
       'windspeed60', 'atmospherepressure60']  
p2=['newtemp90', 'newprecip90', 'newrel_humidity90', 'newwind_dir90',
       'windspeed90', 'atmospherepressure90']       
p3=['newtemp120', 'newprecip120', 'newrel_humidity120', 'newwind_dir120',
       'windspeed120', 'atmospherepressure120'] 
from sklearn.decomposition import PCA, FastICA
pca = PCA(random_state=42,n_components=1)
train_pca = pca.fit_transform(X[p0])
X['pca_0'] = train_pca[:,0]
test_pca = pca.fit_transform(test[p0])
test['pca_0'] = test_pca[:,0]
train_pca = pca.fit_transform(X[p1])
X['pca_1'] = train_pca[:,0]
test_pca = pca.fit_transform(test[p1])
test['pca_1'] = test_pca[:,0]
train_pca = pca.fit_transform(X[p2])
X['pca_2'] = train_pca[:,0]
test_pca = pca.fit_transform(test[p2])
test['pca_2'] = test_pca[:,0]

train_pca = pca.fit_transform(X[p3])
X['pca_3'] = train_pca[:,0]
test_pca = pca.fit_transform(test[p3])
test['pca_3'] = test_pca[:,0]

train_pca = pca.fit_transform(X[p4])
X['pca_4'] = train_pca[:,0]
test_pca = pca.fit_transform(test[p4])
test['pca_4'] = test_pca[:,0]

test=test.drop(columns=['pca_2','pca_4'])
X=X.drop(columns=['pca_4','pca_2'])

In [0]:
X=X.drop(columns=['newtemp105', 'newprecip105', 'newrel_humidity105', 'newwind_dir105',
       'windspeed105', 'atmospherepressure105','newtemp9', 'newprecip9', 'newrel_humidity9', 'newwind_dir9',
       'windspeed9', 'atmospherepressure9'])
test=test.drop(columns=['newtemp105', 'newprecip105', 'newrel_humidity105', 'newwind_dir105',
       'windspeed105', 'atmospherepressure105','newtemp9', 'newprecip9', 'newrel_humidity9', 'newwind_dir9',
       'windspeed9', 'atmospherepressure9']) 

## **hana & and hanat are the data generated by arima ** 
## PLEASE CHECK THAT YOU'VE UPLOAD IT 

In [0]:
hana=pd.read_csv('/content/winddirforecasttrain ADD THOSE COLUMNS TO TRAIN.csv')

In [0]:
hanat=pd.read_csv('/content/winddirforecasttest ADD THOSE FEATURES TO TEST.csv')

In [0]:
hanat.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,197.250658,212.394441,219.223270,222.302612,223.691189,224.317343,224.599697,224.727019,224.784433,224.810323,224.821998,224.827262,224.829636,224.830707,224.831189,224.831407,224.831505,224.831549
1,146.588427,165.851197,176.443787,182.268648,185.471737,187.233115,188.201695,188.734318,189.027206,189.188266,189.276832,189.325535,189.352316,189.367044,189.375142,189.379595,189.382044,189.383391
2,165.153288,165.153288,165.153288,165.153288,165.153288,165.153288,165.153288,165.153288,165.153288,165.153288,165.153288,165.153288,165.153288,165.153288,165.153288,165.153288,165.153288,165.153288
3,268.745697,236.704786,220.128369,211.552532,207.115808,204.820462,203.632961,203.018605,202.700768,202.536334,202.451263,202.407252,202.384483,202.372703,202.366609,202.363456,202.361825,202.360981
4,185.847258,185.847258,185.847258,185.847258,185.847258,185.847258,185.847258,185.847258,185.847258,185.847258,185.847258,185.847258,185.847258,185.847258,185.847258,185.847258,185.847258,185.847258


In [0]:
for i in hana.columns :
  X[i]=hana[i]
for i in hanat :
  test[i]=hanat[i]

In [0]:
X.shape,test.shape

((15539, 774), (5035, 775))

In [0]:
import xgboost as xgb

params = {
            'gpu_id': 0, 
            #'n_gpus': 2, 
            'objective': 'reg:squarederror', 
            'eval_metric': 'rmse', 
             
            'booster': 'gbtree', 
            
            'n_estimators': 10000, 
            'tree_method': 'gpu_hist', 
            'grow_policy': 'lossguide', 
            'max_depth': 8, 
            
             
            
            'learning_rate': 0.01, 
            'max_bin': 200, 
            'max_leaves': 200, 
            
            'reg_alpha': 10, 
            'reg_lambda': 50, 
            'subsample': 0.9 ,
          }
model = xgb.XGBRegressor(**params)

In [0]:
model.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eval_metric='rmse',
             gamma=0, gpu_id=0, grow_policy='lossguide', importance_type='gain',
             learning_rate=0.01, max_bin=200, max_delta_step=0, max_depth=8,
             max_leaves=200, min_child_weight=1, missing=None,
             n_estimators=10000, n_jobs=1, nthread=None,
             objective='reg:squarederror', random_state=0, reg_alpha=10,
             reg_lambda=50, scale_pos_weight=1, seed=None, silent=None,
             subsample=0.9, tree_method='gpu_hist', verbosity=1)

In [0]:
pp=model.predict(test.drop(["ID"],axis = 1))

In [0]:
d = {'ID': test_id, 'target': pp}
subxgb = pd.DataFrame(data=d)
subxgb = subxgb[['ID', 'target']]
subxgb.target.min(),subxgb.target.max()

(15.86525821685791, 238.6833038330078)

In [0]:
subxgb.to_csv('subxgb.csv',index=False)
from google.colab import files
files.download("subxgb.csv")